In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df  = pd.read_csv(r"C:\Users\gagan soni\Downloads\BollywoodMovieDetail.csv")
df.head()

,imdbId,title,releaseYear,releaseDate,genre,writers,actors,directors,sequel,hitFlop
0,tt0118578,Albela,2001,20 Apr 2001,Romance,Honey Irani (screenplay) | Honey Irani (story)...,Govinda | Aishwarya Rai Bachchan | Jackie Shro...,Deepak Sareen,0.0,2
1,tt0169102,Lagaan: Once Upon a Time in India,2001,08 May 2002,Adventure | Drama | Musical,Ashutosh Gowariker (story) | Ashutosh Gowarike...,Aamir Khan | Gracy Singh | Rachel Shelley | Pa...,Ashutosh Gowariker,0.0,6
2,tt0187279,Meri Biwi Ka Jawab Nahin,2004,02 Jul 2004,Action | Comedy,NaN,Akshay Kumar | Sridevi | Gulshan Grover | Laxm...,Pankaj Parashar | S.M. Iqbal,0.0,1
3,tt0222024,Hum Tumhare Hain Sanam,2002,24 May 2002,Drama | Romance,K.S. Adiyaman | Arun Kumar (assistant dialogue...,Shah Rukh Khan | Madhuri Dixit | Salman Khan |...,K.S. Adiyaman,0.0,4
4,tt0227194,One 2 Ka 4,2001,30 Mar 2001,Action | Comedy | Drama,Sanjay Chhel | Raaj Kumar Dahima (screenplay) ...,Shah Rukh Khan | Juhi Chawla | Jackie Shroff |...,Shashilal K. Nair,0.0,1


In [4]:
df.columns

Index(['imdbId', 'title', 'releaseYear', 'releaseDate', 'genre', 'writers',
       'actors', 'directors', 'sequel', 'hitFlop'],
      dtype='object')

In [5]:
df.shape

(1284, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   imdbId       1284 non-null   object 
 1   title        1284 non-null   object 
 2   releaseYear  1284 non-null   int64  
 3   releaseDate  1231 non-null   object 
 4   genre        1282 non-null   object 
 5   writers      1165 non-null   object 
 6   actors       1281 non-null   object 
 7   directors    1280 non-null   object 
 8   sequel       1281 non-null   float64
 9   hitFlop      1284 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 100.4+ KB


## data preprocessing

In [7]:
df.isnull().sum()

imdbId           0
title            0
releaseYear      0
releaseDate     53
genre            2
writers        119
actors           3
directors        4
sequel           3
hitFlop          0
dtype: int64

In [8]:
df.columns

Index(['imdbId', 'title', 'releaseYear', 'releaseDate', 'genre', 'writers',
       'actors', 'directors', 'sequel', 'hitFlop'],
      dtype='object')

In [9]:
df.drop(columns = ['imdbId', 'sequel','writers', 'hitFlop','releaseDate','releaseYear'],inplace = True)

In [10]:
df.shape

(1284, 4)

In [11]:
df.dropna(inplace= True)

In [12]:
df.isnull().sum()

title        0
genre        0
actors       0
directors    0
dtype: int64

In [13]:
df.duplicated().sum()

np.int64(0)

In [14]:
df.head(1)

,title,genre,actors,directors
0,Albela,Romance,Govinda | Aishwarya Rai Bachchan | Jackie Shro...,Deepak Sareen


In [15]:
df['tags'] = df['actors']+df['genre']+df['directors']

In [16]:
df.drop(columns = ['actors','genre','directors'],inplace = True)

In [17]:
 df.tags.iloc[910]

'Ashutosh Rana | Divya Dutta | Rajit Kapoor | Yashpal SharmaDrama | ThrillerSushen Bhatnagar'

In [18]:
import re

In [19]:
df = df.reset_index(drop=True)

In [20]:
for i in range(len(df)):
    df.loc[i, 'tags'] = re.sub('[^a-zA-Z\s]', '', df.loc[i, 'tags'])

In [21]:
# converting the text data to feature vectors
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features_vectors =  vectorizer.fit_transform(df['tags'])

In [22]:
vectorizer.get_feature_names_out()

array(['aabhas', 'aadil', 'aakash', ..., 'zulfi', 'zutshi', 'zutshidrama'],
      shape=(4054,), dtype=object)

In [23]:
len(vectorizer.get_feature_names_out())

4054

In [24]:
y = pd.DataFrame(vectorizer.fit_transform(df['tags']).toarray(),columns = vectorizer.get_feature_names_out())

In [25]:
X = df.title

In [26]:
# getting the similarity scores using cosine similarity

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(features_vectors)

print(similarity)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.07158264]
 [0.         0.         0.         ... 0.         0.07158264 1.        ]]


In [27]:
similarity.shape

(1275, 1275)

In [28]:
list_of_all_titles = df['title'].tolist()

### Getting the movie name  from the user

In [29]:
movie_name = input('enter your fevourite movie name : ')

enter your fevourite movie name :  Mumbai Delhi Mumbai


In [30]:
# finding the close math for the movie name given by the user

import difflib
find_lose_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_lose_match)

['Mumbai Delhi Mumbai', 'Mumbai Diaries', 'Mumbai Meri Jaan']


In [31]:
close_match = find_lose_match[1]
print(close_match)

Mumbai Diaries
